In [327]:
import os
import email
import base64
import numpy as np
import sklearn 

from bs4 import BeautifulSoup as soup

In [328]:
def extract(file_content):
    #d=file_content.split("Subject:")
    #print d[1]
    b = email.message_from_string(file_content)
    if b.is_multipart():
        a=''
        for payload in b.get_payload():
        # if payload.is_multipart(): ...
             a.join(str(payload.get_payload()))
             return a
    else:
        return b.get_payload()
   



In [329]:
def vectorize(body):
    body=body.lower()
    a="hello"
    b="how"
    c="are"
    d="you"
    l=[a,b,c,d]
    a=[0,0,0,0]
    for i,x in enumerate(l):
        if x in body:
            a[i]=1
    return a
    

In [330]:
def create(folder_name):
    
    file_list=os.listdir(folder_name)
    vector=[]
       
    for i in file_list:
        path=os.path.join(folder_name,i)
        f=open(path,'r').read()
        body=extract(f) 
        vector.append(vectorize(body))
    
    return vector
    
    
    
    

In [331]:
def convert(arr):
   
    test=[]
    for i in arr:
        test.append(i[0])
    test=np.asarray(test)
    arr=test
    return arr

In [332]:
spam_vector=np.asarray(create("spam"))
spam_label=np.ones([len(spam_vector),1])

In [333]:
ham_vector=np.asarray(create("easy_ham"))
ham_label=np.zeros([len(ham_vector),1])

type(ham_vector[3])

numpy.ndarray

In [334]:
hard_ham_vector=np.asarray(create("hard_ham"))
hard_ham_label=np.zeros([len(hard_ham_vector),4])

email_data=np.concatenate((spam_vector,ham_vector),axis=0)
#email_data=np.concatenate((spam_vector,ham_vector,hard_ham_vector),axis=0)

email_label=np.concatenate((spam_label,ham_label),axis=0)



In [335]:
from sklearn.model_selection import train_test_split

train_set,test_set=train_test_split(email_data,test_size=0.1,random_state=42)
train_label,test_label=train_test_split(email_label,test_size=0.1,random_state=42)

In [336]:
test_label=convert(test_label)
train_label=convert(train_label)




In [357]:
def train(lf):
    print(lf.fit(train_set,train_label.ravel()))
    return accuracy_score(test_label,lf.predict(test_set))
    

In [358]:
from sklearn.neighbors import KNeighborsClassifier
knn=KNeighborsClassifier()
print(train(knn))

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
0.794117647059


In [359]:
from sklearn.naive_bayes import GaussianNB as GNB
nbc=GNB()
print(train(nbc))

GaussianNB(priors=None)
0.794117647059


In [360]:
from sklearn import svm
svc=svm.SVC()
print(train(svc))


SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape=None, degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)
0.794117647059


In [375]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(train_set, train_label)


RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_split=1e-07, min_samples_leaf=1,
           min_samples_split=2, min_weight_fraction_leaf=0.0,
           n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
           verbose=0, warm_start=False)

In [383]:
from sklearn.model_selection import GridSearchCV
param_grid = [
{'C':[0.010,00001,0.5,0.1],'cache_size':[10,50,100,200,300],'degree':[1,2,3,4,5]}

]
clf=GridSearchCV(svc,param_grid,cv=5,scoring="neg_mean_squared_error")


In [ ]:
clf.fit(train_set,train_label)

In [ ]:
train(clf)